In [76]:
from pinecone import Pinecone
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from pinecone import Pinecone
import itertools
import pandas as pd
import ast
import os
import boto3
from sec_cik_mapper import StockMapper
from pathlib import Path
import pickle
from langchain_chroma import Chroma
from langchain_aws import BedrockEmbeddings
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter


import os
import pandas as pd
from operator import add
from typing import Literal
from typing import Annotated
from pydantic import BaseModel
from dotenv import load_dotenv
from langchain.schema import Document
from typing_extensions import TypedDict
from google.oauth2 import service_account
from langchain_community.vectorstores import FAISS
from langgraph.graph import END, StateGraph, START
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import DataFrameLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.vectorstores.base import VectorStoreRetriever


In [ ]:
os.environ['TAVILY_API_KEY'] = "API-KEY"
os.environ['AWS_ACCESS_KEY_ID'] = "API-KEY"
os.environ['AWS_SECRET_ACCESS_KEY'] = "API-KEY"
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1' 
os.environ['Pinecone_Key'] = "API-KEY"
pc = Pinecone(api_key=os.environ['Pinecone_Key'])

In [78]:
pc.create_index(
  name="research",
  dimension=1024,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

{
    "name": "research",
    "metric": "cosine",
    "host": "research-d3db3ql.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null
}

In [79]:
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-west-2'
)

bedrock_embeddings = BedrockEmbeddings(
    client=bedrock_client, region_name='us-west-2', model_id='amazon.titan-embed-text-v2:0'
)

In [80]:
index = pc.Index('research')

In [81]:
df = pd.read_csv('EarningsCallTranscript.csv')

In [82]:
df

,Company Ticker,Response,CIK,Name,Exchange
0,AAPL,"{""date"": ""2024-10-31"", ""transcript"": ""Suhasini...",320193.0,Apple Inc.,Nasdaq
1,MSFT,"{""date"": ""2024-07-30"", ""transcript"": ""Operator...",789019.0,Microsoft Corp,Nasdaq
2,AMZN,"{""date"": ""2025-02-06"", ""transcript"": ""Operator...",1018724.0,Amazon Com Inc,Nasdaq
3,NVDA,"{""date"": ""2024-02-21"", ""transcript"": ""Operator...",1045810.0,Nvidia Corp,Nasdaq
4,GOOGL,"{""date"": ""2025-02-04"", ""transcript"": ""Operator...",1652044.0,Alphabet Inc.,Nasdaq
...,...,...,...,...,...
551,D,"{""date"": ""2025-02-12"", ""transcript"": ""Operator...",715957.0,"Dominion Energy, Inc",NYSE
552,DTE,"{""date"": ""2025-02-13"", ""transcript"": ""Operator...",936340.0,Dte Energy Co,NYSE
553,DUK,"{""date"": ""2025-02-13"", ""transcript"": ""Elliot: ...",1326160.0,Duke Energy Corp,Nasdaq
554,EIX,"{""date"": ""2025-02-27"", ""transcript"": ""Operator...",827052.0,Edison International,NYSE


In [64]:
def chunks(iterable, batch_size=200):
    """Break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))



,Company Ticker,Response,CIK,Name,Exchange


In [83]:
for i in range(len(df)):
    try:
        d = dict(df.iloc[i])
        name = d["Name"]
        print(name)
        dict_obj = ast.literal_eval(d['Response'])
        date = dict_obj["date"]
        companyTicker = d["Company Ticker"]
        exchange = d["Exchange"]
        CIK = str(dic[companyTicker])
        metadata = {"name" :name, "date": date, "companyTicker":companyTicker, "exchange":exchange, "CIK":CIK }
        transcript = dict_obj["transcript"]


        docs = [Document(page_content=transcript, metadata=metadata)]
        splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
        chunked_docs = splitter.split_documents(docs)

        page = 1

        for batch in chunks(chunked_docs, batch_size=200):
            texts = [doc.page_content for doc in batch]
            metadata_list = [doc.metadata for doc in batch]
            tag = ["EarningsCallTranscript" for doc in batch ]
            embeddings = bedrock_embeddings.embed_documents(texts)

            upsert_data = []

            for i, embedding in enumerate(embeddings):
                upsert_data.append({
                    "id": str(name)+str(page),  
                    "values": embedding,  
                    "metadata": {
                        "name": metadata_list[i]["name"], 
                        "date": metadata_list[i]["date"], 
                        "companyTicker": metadata_list[i]["companyTicker"], 
                        "exchange": metadata_list[i]["exchange"], 
                        "CIK": metadata_list[i]["CIK"], 
                        "text": texts[i] , 
                        "tag": tag[i]
                    }
                })
                page +=1

            index.upsert(vectors=upsert_data)
    except Exception as e:
        print(f"failed for: "+ str(df.iloc[i]["Company Ticker"]))
        continue

    

print("Upsert complete.")


Apple Inc.
Microsoft Corp
Amazon Com Inc
Nvidia Corp
Alphabet Inc.
Alphabet Inc.
Meta Platforms, Inc.
nan
failed for: BRK.B
Tesla, Inc.
Unitedhealth Group Inc
Exxon Mobil Corp
Johnson & Johnson
Jpmorgan Chase & Co
Visa Inc.
Eli Lilly & Co
Walmart Inc.
Procter & Gamble Co
Mastercard Inc
Home Depot, Inc.
Merck & Co., Inc.
Chevron Corp
Abbvie Inc.
Costco Wholesale Corp /New
Oracle Corp
Pepsico Inc
Broadcom Inc.
Thermo Fisher Scientific Inc.
Mcdonalds Corp
Accenture Plc
Danaher Corp /De/
Advanced Micro Devices Inc
Linde Plc
Verizon Communications Inc
Honeywell International Inc
Morgan Stanley
Nextera Energy Inc
Texas Instruments Inc
Amgen Inc
Union Pacific Corp
Rtx Corp
Qualcomm Inc/De
International Business Machines Corp
United Parcel Service Inc
Schwab Charles Corp
Philip Morris International Inc.
Blackrock, Inc.
Caterpillar Inc
S&P Global Inc.
American Tower Corp /Ma/
Boeing Co
Citigroup Inc
Deere & Co
Lowes Companies Inc
Prologis, Inc.
Gilead Sciences, Inc.
Booking Holdings Inc.
nan
fa